In [ ]:
from risk_data import get_factor_data
factor_data = get_factor_data()

In [ ]:
from risk_stats import get_zscore

factor_data['zscore'] = get_zscore(factor_data.ret, factor_data.vol)
factor_data['zscore']

In [ ]:
# sample = factor_data['zscore'].sel(vol_type=21, factor_name='QQQ').to_pandas().dropna()
# plot_qq(sample)

In [ ]:
from risk_chart import plot_qq_df
from risk_data import get_factor_data
from risk_stats import get_zscore

factor_data = get_factor_data()
factor_data['zscore'] = get_zscore(factor_data.ret, factor_data.vol)

df = factor_data['zscore'].sel(vol_type=63, factor_name=['SPY', 'QQQ', 'IWM']).to_pandas()
plot_qq_df(df)


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

def qq_plot2(df: pd.DataFrame, title: str = "QQ Plot") -> go.Figure:
    """
    Create a QQ-plot for each column in a DataFrame using Plotly.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame where each column is a data series to compare against the normal distribution.
        The index should be datetime-like, used for hover labels.
    title : str, optional
        Title of the plot, by default "QQ Plot"
        
    Returns
    -------
    go.Figure
        A Plotly Figure object with QQ plots.
    """
    # Drop any rows with missing data
    clean_df = df.dropna()
    
    # Number of observations
    n = len(clean_df)
    # Theoretical quantiles from standard normal
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.5 / n, 1 - 0.5 / n, n))
    
    fig = go.Figure()

    for col in clean_df.columns:
        sorted_data = clean_df[col].sort_values()
        sorted_dates = clean_df.loc[sorted_data.index].index

        fig.add_trace(go.Scatter(
            x=theoretical_quantiles,
            y=sorted_data.values,
            mode='markers',
            name=col,
            text=[str(d.date()) for d in sorted_dates],
            hovertemplate='%{text}<br>x: %{x:.4f}<br>y: %{y:.4f}<extra></extra>'
        ))

    # Add 45-degree reference line
    q_min = min(theoretical_quantiles.min(), clean_df.min().min())
    q_max = max(theoretical_quantiles.max(), clean_df.max().max())
    fig.add_trace(go.Scatter(
        x=[q_min, q_max],
        y=[q_min, q_max],
        mode='lines',
        line=dict(
            color='rgba(0,0,0,0.3)',
            width=1.5,
            dash='dash'
        ),
        showlegend=False
    ))

    fig.update_layout(
        title=title,
        template='plotly_white',
        width=600,
        height=600,
        xaxis=dict(title='Theoretical Quantiles'),
        yaxis=dict(title='Sample Quantiles'),
    )

    return fig



In [ ]:

from typing import Optional
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as stats


def qq_plot3(
    df: pd.DataFrame,
    dist: str = "norm",
    title: str = "QQ Plot",
    width: int = 600,
    height: int = 600,
) -> go.Figure:
    """
    Generate a QQ-plot comparing columns of a DataFrame to a theoretical distribution.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame where each column is a variable to be plotted.
    dist : str, default "norm"
        Theoretical distribution to compare against. Passed to scipy.stats.
    title : str, default "QQ Plot"
        Title of the plot.
    width : int, default 600
        Width of the figure in pixels.
    height : int, default 600
        Height of the figure in pixels.

    Returns
    -------
    go.Figure
        A Plotly figure object showing the QQ plot.
    """
    df = df.dropna()
    n = len(df)
    if n == 0:
        raise ValueError("Input DataFrame is empty after dropping missing values.")

    probs = np.linspace(0.5 / n, 1 - 0.5 / n, n)
    theoretical_quantiles = getattr(stats, dist).ppf(probs)

    fig = go.Figure()

    for col in df.columns:
        sorted_series = df[col].sort_values()
        sorted_values = sorted_series.values
        sorted_dates = sorted_series.index.strftime("%Y-%m-%d")

        fig.add_trace(
            go.Scatter(
                x=theoretical_quantiles,
                y=sorted_values,
                mode="markers",
                name=col,
                text=sorted_dates,
                hovertemplate=(
                    f"<b>{col}</b><br>"
                    "Date: %{text}<br>"
                    "Theoretical: %{x:.2f}<br>"
                    "Observed: %{y:.2f}<extra></extra>"
                ),
            )
        )

    # Add 45-degree reference line
    all_values = df.values.flatten()
    min_val = np.min(all_values)
    max_val = np.max(all_values)
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode="lines",
            line=dict(color="rgba(204, 204, 204, 1)", width=1),
            showlegend=False,
            hoverinfo="skip",
        )
    )

    fig.update_layout(
        title=title,
        xaxis_title=f"Theoretical Quantiles ({dist})",
        yaxis_title="Observed Values",
        template="plotly_white",
        width=width,
        height=height,
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)

    return fig


In [ ]:
qq_plot2(df)

In [ ]:
qq_plot3(df)

In [ ]:
factor_1 = 'SPY'
vol_type_1 = 63
factor_2 = 'IWM'
vol_type_2 = 63


df = pd.concat({f'{factor_1}, {vol_type_1}' : factor_data.zscore.sel(factor_name=factor_1, vol_type=vol_type_1).to_pandas(),
                f'{factor_2}, {vol_type_2}' : factor_data.zscore.sel(factor_name=factor_2, vol_type=vol_type_2).to_pandas()}
               , axis=1).dropna()

# pd.concat([factor_data.zscore.sel(factor_name=factor_1, vol_type=vol_type_1).to_pandas(),
#            factor_data.zscore.sel(factor_name=factor_2, vol_type=vol_type_2).to_pandas(),
# ], axis=1).dropna()


In [ ]:
factor_data.zscore.sel(factor_name=factor_1, vol_type=vol_type_1).to_series()
factor_data.zscore.sel(factor_name=factor_1, vol_type=vol_type_1).to_pandas()

In [ ]:
n = 273
np.linspace(0.5/n, 1 - 0.5/n, n)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Sample data
np.random.seed(0)
x = np.linspace(-3, 3, 100)
y = np.random.normal(0, 1, size=100)

# Figure with <extra></extra> to suppress the extra box
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=x, y=y, mode='markers',
    name='No Extra',
    hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>"
))
fig1.update_layout(title='Without <extra>')

# Figure with default hovertemplate (includes extra box)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=x, y=y, mode='markers',
    name='Default',
    hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}"  # No <extra>
))
fig2.update_layout(title='With Default <extra>')

fig1.show()
fig2.show()


In [ ]:
import scipy.stats as stats

def get_dist_name(dist) -> str:
    return getattr(dist, "name", type(dist).__name__.replace("_gen", "")).capitalize()

def get_dist_name(dist) -> str:
    return type(dist).__name__.replace("_gen", "").capitalize()

get_dist_name(stats.norm)  # 'norm'

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)

In [ ]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


(-1094.2357165991245, 1451.9756745696477)